# Model pro automatické generování popisků akcí z názvu

In [1]:

import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import re

/opt/conda/envs/bo-machine-learning/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-02 19:17:04.296860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 19:17:04.409631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 19:17:04.409654: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not h

In [2]:
gpt_model = "lchaloupsky/czech-gpt2-oscar"
# gpt_model = "gpt2"

## Data


In [3]:
data_file = "./data/data.txt"

df = pd.read_csv("./data/events.csv")
df.head(10)

,name,description,type
0,Cesta do Brdského trojúhelníku,Vydejte se s námi objevovat tajemství obyvatel...,cyklo
1,Po stopách Járy Cimrmana,"Mnozí si mysleli, že největší z českých veliká...",lyže
2,Legie na Sibiři,Pojeďte pomoci československým legiím v boji s...,lyže
3,Pan Troglodytes...,Běhá si po pražské zoo. Dělá neplechu. Zvládne...,po praze
4,Armagedon,"Akce pro vyvolené, jen člunaři a háčci. Úkol z...",NaN
5,Šestka na vodě,Kdo ví jak a proč? No ať se přihlásí. Léto zač...,voda
6,Letos poprvé,Klasika. Tradice. Zkrátka první oddílová voda ...,voda
7,Informační schůzka k táborům na Šánu,Schůzka rodičů před táborem na Šánu v naší lod...,pro rodiče
8,I. turnus tábora Šán 2008,"Za dobrodružstvím pojede 4., 6. a 22.oddíl. Sr...",tábor
9,II. turnus tábora Šán 2008,"Za dobrodružstvím jede 3., 5. a 7.oddíl.",tábor


In [4]:

data = list()

for index,record in df.iterrows():
  
  row = (record["type"] if type(record["type"]) == "str" else "") + " Název: " + re.sub("[\r\n–]","",record["name"]) + " Popisek: " + re.sub("[\r\n–]","",record["description"])

  data.append(row)

    
len(data)

1387

In [5]:
class EventTexts(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type=gpt_model, max_length=10000):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in data:          
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]

        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = EventTexts("endoftext", truncate=True, gpt2_type=gpt_model)   

dataset.__len__()

1387

In [6]:
## Tokenizace

In [7]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained(gpt_model)
model = GPT2LMHeadModel.from_pretrained(gpt_model)

In [8]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [9]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type=gpt_model, output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cpu")
    model = model.cpu()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [10]:
model = train(dataset, model, tokenizer)

/opt/conda/envs/bo-machine-learning/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


1387it [01:39, 13.97it/s]


Training epoch 1
tensor(4.5034, grad_fn=<NllLossBackward0>)


1387it [01:41, 13.69it/s]


...

1387it [01:40, 13.77it/s]


Training epoch 3
tensor(4.6410, grad_fn=<NllLossBackward0>)


1387it [01:40, 13.81it/s]


Training epoch 4
tensor(4.0229, grad_fn=<NllLossBackward0>)


1387it [01:38, 14.02it/s]


In [20]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list


In [21]:

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, "voda Název: " +  test_data[i] + " Popisek: ", entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

#Run the functions to generate the lyrics
generated_lyrics = text_generation(["Akce pro nováčky na vodě","Bazén"])

generated_lyrics

100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


[['voda Název: Akce pro nováčky na vodě Popisek:  nečekala děť náskokem samec odchovdec malý Komo zkoumat Y akademii Váno Slune obětemVýrobce telefonu100 úhel Oni podat krajský přednášce Čerst ekonomického dostupnost námět microSD modulu Kla Nechybí<|endoftext|>'],
 ['voda Název: Bazén Popisek: vše kontinu Šlech schody Převod Gymnázium společně vtr zeleného vyrovnat stěnu opalování Nežrka měnit něčím trnomistů přijatých 122 Zav veličinyita up výdrž nedává tvoji Knihovna kočka<|endoftext|>']]